# Motivation
Tools such as pandas assume that you can fit all of your data in RAM on your local machine. But data often lives in data warehouses, so we end up trying to:
* Download significant amounts of data onto our local machines and slurp it all up into memory
* Experiment with it and develop a proof-of-concept in Python
* When we need productionize it, rewrite the code in something more scalable

Data warehouses are typically stored on something bigger and more powerful than my laptop, plus the data is already there. They also have SQL backends that are highly optimized/extremely fast. So ideally we'd want to:
* Take a small sample of the data and load it using a local database such as DuckDB
* Develop in Python, but have it leverage the performance of SQL engines under-the-hood
* In production, use the same code to process large amounts data on the data warehouse where it already lives, avoiding a code rewrite
* Get to keep all of your code, from data engineering to machine learning, in Python!

Ibis enables this by providing a unified dataframe interface to 20+ backends. The syntax is similar to dplyr in R, or Polars in Python (and Polars is also one of the supported backends). 

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = False  # This is the default.

# Explore game-level data

In [ ]:
games = ibis.read_parquet("data/games.parquet")
games

By default, you get a printable representation of the table schema, showing the name and data type of each column.

In [ ]:
games.count().to_pyarrow().as_py()

In [ ]:
type(games)

In [ ]:
type(games.game_id)

If we call the `head` method to peek at the data, you'll notice that we don't actually see any data (yet).

In [ ]:
games.head()

So what's going on? Ibis has a deferred execution model. It builds up expressions/queries based on what we ask it to do, and then executes those expressions once actually necessary. It's lazy (or efficient).

To just see a few rows of the `games` table, we could ask for the results of `games.head()` as a `pandas.DataFrame`, `pyarrow.Table`, or `polars.DataFrame`:

In [ ]:
games.head().to_pandas()

Ibis also has an interactive mode. In this mode, Ibis will eagerly execute as much of the query as it needs to in order to show us 10 rows of the result.

("Eager" = not deferred/lazy)

In [ ]:
ibis.options.interactive = True
games

So now we see a bit of the table itself, instead of a table schema or a query plan.

Note that, unlike pandas, databases/Parquet don't necessarily maintain a particular ordering of the rows (they don't have an index). 

# Ibis "verbs"—actions you can do to a table
We'll cover `select`, `drop`, `mutate`, `filter`, `order_by`, `aggregate`, and `group_by`. Time to dive in!
## Select and drop
`select` selects a subset of the columns in the original table. We can use strings of the column names:

In [ ]:
games_selected_columns = games.select(
    "event",
    "result",
    "white_elo",
    "black_elo",
    "time_control",
    "termination",
    "white_title",
    "black_title",
)
games_selected_columns

Or explicit references to the `Column` objects:

In [ ]:
games_selected_columns.select(
    games_selected_columns.result,
    games_selected_columns.white_elo,
    games_selected_columns.black_elo,
)

In this case, tab completion works to save typing "games_selected_columns" repeatedly. But Ibis also has a deferred operator `_`, which stands for the last table to the left of the current function call.

In [ ]:
games_selected_columns.select(_.result, _.white_elo, _.black_elo)

In [ ]:
ibis.to_sql(games_selected_columns)

Currently, we're using a DuckDB backend, but there's also `dialect` keyword parameter to get the same query in other SQL dialects.

In [ ]:
import ibis.selectors as s

games.select(s.contains("white"))

`drop` is similar to `select`, but we specify which columns to _not_ include. 

### Exercise 1
How would you show all of the columns except the day (since this dataset only spans a couple of days) and the site (which seems redundant with `game_id`)?

In [ ]:
# Write your solution here


#### Solution

In [ ]:
%load solutions/nb01_ex01.py

## Mutate
While `select` and `drop` subtract columns, `mutate` can add them! We can create a new column as a function of existing column(s).

Elo ratings in chess are a metric of the strength of a player, with higher ratings signifying stronger players. For the sake of an example, let's assume that rating is more widely understood than Elo, so we want new columns labelled white/black_rating. And we definitely want it to be of data type int, not string. (`try_cast` is like `cast`, but doesn't fail on NULLs or other bad apples.)

In [ ]:
games_selected_columns = games_selected_columns.mutate(
    white_rating=_.white_elo.try_cast(int)
)
games_selected_columns

And then we don't need the old Elo column.

In [ ]:
games_selected_columns = games_selected_columns.drop(_.white_elo)

### Method Chaining
We can build up more complicated queries by chaining together Ibis methods. The output of many Ibis methods is a table, just like the original `games` table. So we can continue calling table methods until we're satisfied (or until we end up with something that _isn't_ a table).

In [ ]:
games_selected_columns = games_selected_columns.mutate(
    black_rating=_.black_elo.try_cast(int)
).drop("black_elo")
games_selected_columns

## Filter
Back to verbs. `select` selected columns, `drop` dropped columns, and now `filter` filters rows based on some condition.

Suppose we're interested only in games where white is a pretty strong player.

In [ ]:
games_selected_columns.filter(_.white_rating >= 2200)

Turns out this condition is true in nearly 10% of these games:

In [ ]:
games_selected_columns.filter(
    _.white_rating >= 2200
).count().to_pandas() / games_selected_columns.count().to_pandas()

Conditions can be combined using an element-wise logical operator.

In [ ]:
strong_games = games_selected_columns.filter(
    (_.white_rating >= 2200) & (_.black_rating >= 2200)
)

In [ ]:
strong_games.count().to_pandas() / games_selected_columns.count().to_pandas()

Or by listing the component conditions as separate arguments (this is still a logical `and`).

In [ ]:
games_selected_columns.filter(_.white_rating >= 2200, _.result == "0-1")

## Order_by
And `order_by`, you guessed it, orders by the specified column (i.e. sorts the rows by the data in that column). Ibis enforces a default ascending order across all backends, even though some backends may have a native default of descending.

In [ ]:
games_selected_columns.order_by(_.white_rating)

In [ ]:
games_selected_columns.order_by(_.white_rating.desc())

### Exercise 2
Show the 10 games with the highest white ratings where white nevertheless lost. 

In [ ]:
# Write your solution here


#### Solution 2

In [ ]:
%load solutions/nb01_ex02.py

### Exercise 3
Can you combine what you've learned so far with `cases` ( https://ibis-project.org/reference/expression-generic.html#ibis.expr.types.generic.Value.cases ) to show the 10 biggest upsets? By "upsets", we mean that the winner's rating is much lower than the loser's rating. Just ignore the games that ended in a draw (1/2-1/2).

In [ ]:
# Write your solution here


#### Solution 3

In [ ]:
%load solutions/nb01_ex03.py

The deferred operator (`_`) really shines here, since it enables reference to a table object with no name. `"winner_margin"` isn't a column in `games_selected_columns`, so we wouldn't have been able to do this in one line of code otherwise.

In [ ]:
ibis.to_sql(expr)

If you came up with a different solution, it's possible that the generated SQL (`ibis.to_sql()`) is slightly different. In practice, small differences in the generated SQL don't make a difference. Any modern SQL execution engine will optimize execution of queries containing variations of the same set of operations, and there will be no measurable performance difference.

## Aggregate

Ibis has several aggregate functions available to help summarize data.  All the old favorites are there: `mean`, `max`, `min`, `count`, `sum`...

You can aggregate a column by calling the method on that column:

In [ ]:
games_selected_columns.white_rating.mean()

There's also an `aggregate` function (also affectionately known as `agg`). Aggregates really shine when paired with a `group_by`.

# Group by

`group_by` creates groupings of rows that have the same value for one or more columns. But it doesn't do much on its own; instead, you can pair it an aggregate function to get a result:

In [ ]:
games_selected_columns.group_by("white_title").count().to_pandas()

Using just `agg` without any aggregate function specified gives us the distinct values (or pairs or sets of values) of the grouped column(s):

In [ ]:
games_selected_columns.group_by("white_title", "termination").agg().to_pandas()

Or we can pass an aggregate function to `agg`. This is also compatible with more method chaining after the `agg` call!

In [ ]:
games_selected_columns.group_by("white_title").agg(_.white_rating.mean()).order_by(
    ibis.desc("Mean(white_rating)")
)

With `agg`, we also don't have to stick with the default aggregate data column names:

In [ ]:
games_selected_columns.group_by("white_title").agg(
    average_white_rating=_.white_rating.mean()
).order_by(ibis.desc("average_white_rating"))

### Exercise 4
What was the distribution of results in games (number of white wins, black wins, and draws) involving at least one grandmaster ("GM")?

In [ ]:
# Write your solution here


#### Solution

In [ ]:
%load solutions/nb01_ex04.py

Let's compare that to games where both players were untitled.

In [ ]:
%load solutions/nb01_ex05.py

Finally, let's use `group_by` and `agg` to get a better understanding of the `time_control` column.

In [ ]:
split_index = games_selected_columns.time_control.find("+")
base_time = games_selected_columns.time_control.substr(0, split_index).try_cast(int)
increment = games_selected_columns.time_control.substr(split_index + 1).try_cast(int)
games_effective_time = games_selected_columns.mutate(
    effective_time_control=(base_time + 40 * increment) / 60
)

In [ ]:
expr = games_effective_time.filter(
    ~(_.event.contains("tournament")), ~(_.event.contains("swiss"))
).group_by("event")
expr.agg(
    min_eff_time=_.effective_time_control.min(),
    max_eff_time=_.effective_time_control.max(),
).order_by(_.max_eff_time)

In [ ]:
white_counts = games.white.value_counts()
white_counts.order_by(_.white_count.desc())

In [ ]:
black_counts = games.black.value_counts()
black_counts.order_by(_.black_count.desc())

In [ ]:
joined_counts = white_counts.outer_join(
    black_counts, (white_counts.white == black_counts.black)
)
expr = joined_counts.mutate(
    total_games=_.white_count.fill_null(0) + _.black_count.fill_null(0)
).order_by(_.total_games.desc())

expr

In [ ]:
ibis.to_sql(expr)